# Classificació de text segons l'autor
  Construïu dos models de diferents autors: Josep Carner i Miquel dels Sants Oliver. Per això, podeu usar les seves obres disponibles al Projecte Gutenberg. Per exemple, de Carner preneu-ne la traducció dels contes de Mark Twain. Després, classificau frases en l'estil de cadascú que mostrin com el vostre model les identifica correctament.

In [3]:
!pip install -r requirements.txt

  Using cached https://github.com/explosion/spacy-models/releases/download/ca_core_news_sm-3.8.0/ca_core_news_sm-3.8.0-py3-none-any.whl (19.6 MB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
ERROR: Ignored the following versions that require a different python version: 1.2.0 Requires-Python <3.13,>=3.6
ERROR: Could not find a version that satisfies the requirement blis==1.2.0 (from versions: 0.0.1, 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.8, 0.0.9.dev104, 0.0.10, 0.0.12, 0.0.13, 0.0.16, 0.1.0, 0.2.0.dev0, 0.2.0, 0.2.1, 0.2.2.dev0, 0.2.2, 0.2.3.dev0, 0.2.3.dev1, 0.2.3.dev2, 0.2.3.dev3, 0.2.3, 0.2.4, 0.3.1, 0.4.0.dev0, 0.4.0.dev1, 0.4.0, 0.4.1, 0.7.0, 0.7.1.dev0, 0.7.1, 0.7.2, 0.7.3, 0.7.4, 0.7.5, 0.7.6, 0.7.7, 0.7.8, 0.7.9, 0.7.10, 0.7.11, 0.9.0, 0.9.1.dev0, 0.9.1.dev1, 0.9.1, 1.0.0a1, 1.0.0, 1.0.1, 1.0.2, 1.1.0a0, 1.1.0)

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution 

In [4]:
from aima_data.utils import open_data
from aima_data.text import words, UnigramWordModel
from aima_data.learning import NaiveBayesLearner

def recognize_author(text, nbs):
   return nbs(words(text.lower()))

def create_unigram_model(file_path, n):
   text = open_data(file_path).read()
   wordseq = words(text)
   return UnigramWordModel(wordseq, n)

p_miquel = create_unigram_model('CA-Text/bolla_hostel.txt', 5)
p_josep = create_unigram_model('CA-Text/tom_sawyer_adventures.txt', 5)

nbs = NaiveBayesLearner({('Miquel', 1): p_miquel, ('Josep', 1): p_josep}, simple=True)


ModuleNotFoundError: No module named 'numpy'

![image.png](image.png)

Ara tenim les dues frases següents:
De Miquel dels Sants Oliver:
"La història és un diàleg entre el passat i el present, una constant revisió de criteris i valors."
De Josep Carner:
"Les paraules són com ocells que volen de branca en branca fins a trobar el seu cant definitiu."

In [ ]:
recognize_author('entre el passat i el present, una constant revisió de criteris i valors.')
recognize_author('ocells que volen de branca en branca fins a trobar el seu cant definitiu.')

# Generació de text
Construïu models n-gram amb n=1, n=3, n=5 i n=7 a partir d'un text triat per vosaltres. Pot ser una obra del Projecte Gutenberg o una notícia de premsa, per exemple. Observau com a mesura que augmenta el nombre de paraules que s'hi han tingut en compte la versemblança del text generat és més gran.

In [ ]:
from random import choice
from aima_data.text import NgramCharModel, words

# generate text funcion
def generate_text(model, n, ngram=5):
    generate = []
    seed = choice(list(model.dictionary.keys()))
    generate.extend(seed)

    for i in range(n - ngram):
        last = tuple(generate[-(ngram - 1):])
        next_chars = [j[ngram - 1] for j in model.dictionary.keys() if j[:ngram - 1] == last]

        if not next_chars:
            break

        next_char = choice(next_chars)
        generate.append(next_char)

    return ' '.join(generate)

bollaHostel = open_data('CA-Text/bolla_hostel.txt').read()
wordseq = words(bollaHostel)

for i in range (1, 6):
    model = NgramCharModel(wordseq, i)
    print(generate_text(model, 100))
    print(f'Ngram: {i}\n')
    print({generate_text(model, 100)})
    print('\n')


# Anàlisi sintàctica
Aplicau les eines d'alguna llibreria Python per analitzar sintàcticament l'oració següent: Tots els éssers humans neixen lliures i iguals en dignitat i drets.

In [ ]:
!python -m spacy download ca_core_news_sm
!python -m spacy download es_core_news_sm

In [ ]:
import spacy

es_nlp = spacy.load('es_core_news_sm')
ca_nlp = spacy.load('ca_core_news_sm')

es_sentence = 'Todos los seres humanos nacen libres e iguales en dignidad y derechos'
ca_sentence = 'Tots els éssers humans neixen lliures i iguals en dignitat i drets'

es_doc = es_nlp(es_sentence)
ca_doc = ca_nlp(ca_sentence)

In [ ]:
print('- CATALÀ -')
print('Noun Phrases:', [chunk.text for chunk in ca_doc.noun_chunks])
print('Verbs:', [token.lemma_ for token in ca_doc if token.pos_ == 'VERB'])
print('Subjects:', [token.text for token in ca_doc if token.dep_ == 'nsubj'])
print('Adjectius:', [token.text for token in ca_doc if token.pos_ == 'ADJ'])
print('Preposicions:', [token.text for token in ca_doc if token.pos_ == 'ADP'])
print('Conjuncions:', [token.text for token in ca_doc if token.pos_ == 'CCONJ'])
print('+ Informació:')
for token in ca_doc:
    print(f'Paraula: {token.text} | Lema: {token.lemma_} | Part del discurs: {token.pos_} | Dependència sintàctica: {token.dep_} | Head: {token.head.text}')

In [ ]:
print('- CASTELLANO -')
print('Noun Phrases:', [chunk.text for chunk in es_doc.noun_chunks])
print('Verbos:', [token.lemma_ for token in es_doc if token.pos_ == 'VERB'])
print('Sujetos:', [token.text for token in es_doc if token.dep_ == 'nsubj'])
print('Adjetivos:', [token.text for token in es_doc if token.pos_ == 'ADJ'])
print('Preposiciones:', [token.text for token in es_doc if token.pos_ == 'ADP'])
print('Conjunciones:', [token.text for token in es_doc if token.pos_ == 'CCONJ'])
print('+ Información:')
for token in es_doc:
    print(f'Palabra: {token.text} | Lema: {token.lemma_} | Parte del discurso: {token.pos_} | Dependencia sintáctica: {token.dep_} | Head: {token.head.text}')

# Recuperació d'informació
Carregau l'article de la wikipèdia sobre Europa (https://ca.wikipedia.org/wiki/Europa) en una llista de frases i recuperau-ne la informació següent, cercant les frases més semblants.

Quan es va gestar el concepte d'Europa?
Quina és l'espècie humana autòctona d'Europa?
Quan es varen formar els estats actuals d'Europa?
Quin clima té Europa?

In [ ]:
from requests import get
from bs4 import BeautifulSoup
import spacy
from sentence_transformers import SentenceTransformer, util

# fetch wikipedia content function
def fetch_wikipedia_content(url):
    response = get(url)
    html = BeautifulSoup(response.text, 'html.parser')
    content = ' '.join([p.text for p in html.find_all(["p", "div"]) if p.text])
    return content

# processor text with spacy function
def process_text_with_spacy(content, model_name='ca_core_news_sm'):
    nlp = spacy.load(model_name)
    document = nlp(content)
    sentences = [sentence.text for sentence in document.sents]
    return sentences

# best matching thing
def find_best_matching_sentences(questions, sentences, model_name='paraphrase-multilingual-MiniLM-L12-v2'):
    model = SentenceTransformer(model_name)
    questions_embeddings = model.encode(questions, convert_to_tensor=True)
    sentences_embeddings = model.encode(sentences, convert_to_tensor=True)
    
    best_sentences = []
    for i, question in enumerate(questions):
        scores = util.pytorch_cos_sim(questions_embeddings[i], sentences_embeddings)
        best_sentence_idx = scores.argmax()
        best_sentences.append(sentences[best_sentence_idx])
    return best_sentences

url = 'https://ca.wikipedia.org/wiki/Europa'
content = fetch_wikipedia_content(url)
sentences = process_text_with_spacy(content)

questions = [
    '¿Cuándo se gestó el concepto de Europa?',
    '¿Cuál es la especie humana autóctona de Europa?',
    '¿Cuándo se formaron los estados actuales de Europa?',
    '¿Qué clima tiene Europa?'
]

best_sentences = find_best_matching_sentences(questions, sentences)

for i, question in enumerate(questions):
    print(f'Question {i+1}: {question}\nBest Sentence: {best_sentences[i]}\n\n')